Если в Colab: необходимо включить GPU для более быстрого рендеринга.
Загрузка зависимостей:
* Whisper распознаёт голос и преобразует в txt. Он делает основную магию. Вес 15 МB
* ffmpeg нужен для разбиения динного аудио на части. Обычно в Colab, ffmpeg уже установлен.


In [8]:
!pip install pydub openai-whisper
!pip install pydub
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


Импорт зависимостей. В том числе для управления файлами внутри Colab.

In [14]:
import whisper
import torch # Для автоматического выбора устройства
from tqdm import tqdm
from pydub import AudioSegment # Для сегментации аудио
import os
from google.colab import files # Для Colab обязательно

Далее идёт магия. Длинное аудио будет делиться на чанки по 20 минут, если аудио будет меньшего размера, то код всё равно отработает без ошибок. При выполнении локально, в audio_path необходимо вписать путь до файла с аудиозаписью. Важным моментом является выбор модели:
* small - 435 MB, высокая скорость и малая точность. Можно понять контекст, но не детали
* medium - 1.5 GB, медленнее, однако позволяет точнее понимать суть текста.


In [16]:

def transcribe_long_audio(
    audio_path: str,
    output_txt: str,
    model_name: str = "base",
    chunk_duration_ms: int = 20 * 60 * 1000,  # 20 минут
    overlap_ms: int = 3000
):
    """
    Транскрибирует длинное аудио, разбивая его на чанки.

    :param audio_path: Путь к входному аудиофайлу (.m4a, .mp3, .wav и т.д.)
    :param output_txt: Путь к выходному .txt файлу
    :param model_name: Модель Whisper ('tiny', 'base', 'small', 'medium', 'large')
    :param chunk_duration_ms: Длина каждого чанка в миллисекундах
    :param overlap_ms: Перекрытие между чанками (во избежание обрезания слов)
    :return: None
    """
    # Автоматически выбирается используемое устройство (GPU или CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Используется устройство: {device}")

    try:
        model = whisper.load_model(model_name, device=device)
    except Exception as e:
        print(f"Ошибка при загрузке модели {model_name}: {e}")
        return

    audio = AudioSegment.from_file(audio_path)
    total_ms = len(audio)

    full_text = ""
    start_ms = 0
    chunk_index = 0

    while start_ms < total_ms:
        end_ms = min(start_ms + chunk_duration_ms, total_ms)
        chunk = audio[start_ms:end_ms + overlap_ms]
        chunk_path = f"temp_chunk_{chunk_index}.wav"
        chunk.export(chunk_path, format="wav")

        try:
            result = model.transcribe(chunk_path)
            full_text += result["text"] + " "
        except Exception as e:
            print(f"Ошибка при обработке чанка {chunk_index}: {e}")

        # Удаление временного файла
        if os.path.exists(chunk_path):
            os.remove(chunk_path)

        start_ms += chunk_duration_ms
        chunk_index += 1

    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(full_text.strip())

    print(f"Готово! Результат сохранён в {output_txt}")


# Загрузка аудиофайла
print("Загрузите ваш .m4a файл:")
uploaded = files.upload()
if not uploaded:
    print("Файл не был загружен.")
    exit()

audio_file = list(uploaded.keys())[0]

# Параметры
output_file = "output.txt"
model_name = "small"  # или "base", "small", в зависимости от мощности GPU

# Запуск транскрибации
transcribe_long_audio(
    audio_path=audio_file,
    output_txt=output_file,
    model_name=model_name,
    # chunk_duration_ms и overlap_ms используются по умолчанию
)

# Скачивание результата
print("Скачивание файла...")
files.download(output_file)

Загрузите ваш .m4a файл:


Saving example.m4a to example (1).m4a
Используется устройство: cuda
Готово! Результат сохранён в output.txt
Скачивание файла...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Для аудио, длинной в 2 минуты, при заранее скачанной модели medium, преобразование длится 43 секунды (c учётом времени на загрузку файла). <br>
Для того же файла с моделью small - 25 секунд, причём информативность текста значительно упала.